In [2]:
import numpy as np
import torch.nn as nn
from PIL import Image
import glob
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import pandas as pd
import math

#img = Image.open("284.Bmp", "r")

N = 6283
D_in = 400
D_in2 = 20
H = 100
D_out = 62

x_list = []
x_data = np.zeros([N, D_in])
x_data2 = np.zeros([N, 1, D_in2, D_in2])
y_data = np.zeros([N, D_out])
y_data2 = np.zeros([N])

print("a")

for img_id in range(N + 1):
    for filename in glob.glob("C:\\Users\\ahmetcan\\Documents\\SWE\\SWE 582\\Google Street View\\trainResized\\" 
                              + str(img_id) + ".bmp"):
        im=Image.open(filename)
        pix = np.array(im)
        
        if len(pix.shape) > 2:
            pix2 = pix.mean(2)
            pix3 = pix2.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix2
        else:
            pix3 = pix.reshape((1, D_in))
            x_data2[img_id - 1, 0, :, :] = pix
            
        x_list.append(pix3)
        x_data[img_id - 1] = pix3

print("b")

a
b


In [3]:
def conv_lbl_to_int(a):
    if a.islower():
        return ord(a) - 61
    elif a.isnumeric():
        return int(a)
    else:
        return ord(a) - 55
    
df_labels = pd.read_csv('trainLabels.csv',header=0)

for index, row in df_labels.iterrows():
    if index < N:
        y_data2[index] = conv_lbl_to_int(row['Class'])

y_data2

x_tensor2 = torch.Tensor(x_data2)

y_tensor2 = torch.Tensor(y_data2)
y_tensor2 = y_tensor2.type(torch.LongTensor)

print("x")
y_tensor2

x



 49
  8
 29
 ⋮ 
 25
 23
 27
[torch.LongTensor of size 6283]

In [17]:
## Hyper Parameters
#num_epochs = 2
#batch_size = 100
#learning_rate = 0.001
#
# CNN Model (2 conv layer)
#class CNN(nn.Module):
#    def __init__(self):
#        super(CNN, self).__init__()
#        self.layer1 = nn.Sequential(
#            nn.Conv2d(1, 16, kernel_size=5, padding=2),
#            nn.BatchNorm2d(16),
#            nn.ReLU(),
#            nn.MaxPool2d(2))
#        self.layer2 = nn.Sequential(
#            nn.Conv2d(16, 32, kernel_size=5, padding=2),
#            nn.BatchNorm2d(32),
#            nn.ReLU(),
#            nn.MaxPool2d(2))
#        self.fc = nn.Linear(800, 62)
#        
#    def forward(self, x):
#        out = self.layer1(x)
#        out = self.layer2(out)
#        out = out.view(out.size(0), -1)
#        out = self.fc(out)
#        return out
#        
#cnn = CNN()
#
#
## Loss and Optimizer
#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
#
## Train the Model
#for epoch in range(num_epochs):
#    #for i, (images, labels) in enumerate(train_loader):
#    images = Variable(x_tensor2)
#    labels = Variable(y_tensor2)
#    
#    # Forward + Backward + Optimize
#    optimizer.zero_grad()
#    outputs = cnn(images)
#    loss = criterion(outputs, labels)
#    loss.backward()
#    optimizer.step()
#    
#    print("loss: ", loss.data[0])
#    
#    #if (i+1) % 100 == 0:
#    #    print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
#    #           %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(80, 62)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = torch.optim.SGD(model.parameters(), lr=0.0007, momentum=0.7)

#rnd = np.random.randint(100, size=(N, 1, 3, 3))

#numberOfBatches = 2

batch_size = 600
numberOfBatches = math.ceil(N/batch_size)
idx_start = 0

#for batch_idx in range(numberOfBatches):
#    
#    #print("batch_idx: ", batch_idx)
#    #print("start: ", idx_start, "end: ", idx_start + batch_size)
#    #print("data: ", rnd[idx_start:idx_start + batch_size,:,:,:])
#    #print("shape: ", rnd[idx_start:idx_start + batch_size,:,:,:].shape)
#    idx_start = idx_start + batch_size

#print(x_data2[5:50,:,:,:].shape)
#print(y_data2.shape)
#print(y_data2[5:50].shape)

for epoch in range(1, 500):
    idx_start = 0
    for batch_idx in range(numberOfBatches):
        #train(epoch)
        #model.train()
        #for batch_idx, (data, target) in enumerate(train_loader):
        x_data3 = x_data2[idx_start:idx_start + batch_size,:,:,:]
        #print("batch_idx: ", batch_idx, "x-data3", x_data3)
        y_data3 = y_data2[idx_start:idx_start + batch_size]
        #print("batch_idx: ", batch_idx)
        #print("start: ", idx_start, "end: ", idx_start + batch_size)
        #print("x_data3.shape: ", x_data3.shape)
        #print("y_data3.shape: ", y_data3.shape)
        
        x_tensor3 = torch.Tensor(x_data3)
        #print("batch_idx: ", batch_idx, "x_tensor3", x_tensor3)
        
        y_tensor3 = torch.Tensor(y_data3)
        y_tensor3 = y_tensor3.type(torch.LongTensor)
        
        data, target = Variable(x_tensor3), Variable(y_tensor3)
        #print("batch_idx: ", batch_idx, "target", target)
        optimizer.zero_grad()
        output = model(data)
        #rint("batch_idx: ", batch_idx, "output", output)
        loss = F.nll_loss(output, target)
        loss.backward()
        if epoch % 5 == 0:# and batch_idx == 5:
            print("epoch: ", epoch, "loss_b", loss)
        optimizer.step()
        #print(output)
        #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #    epoch, batch_idx * len(data), batch_size,
        #    100. * batch_idx / batch_size, loss.data[0]))
        idx_start = idx_start + batch_size

epoch:  5 loss_b Variable containing:
 4.1185
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1243
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1259
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1226
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1301
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1191
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1204
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1236
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1220
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1180
[torch.FloatTensor of size 1]

epoch:  5 loss_b Variable containing:
 4.1331
[torch.FloatTensor of size 1]

epoch:  10 loss_b Variable containing:
 4.1148
[torch.FloatTensor of size 1]

epoch:  10 loss_b Variable containing:
 4.1208
[torch.FloatTensor of size 1

epoch:  50 loss_b Variable containing:
 4.0670
[torch.FloatTensor of size 1]

epoch:  50 loss_b Variable containing:
 4.0300
[torch.FloatTensor of size 1]

epoch:  50 loss_b Variable containing:
 4.0767
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0274
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0051
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0728
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0322
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0337
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0295
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 3.9995
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0324
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 4.0285
[torch.FloatTensor of size 1]

epoch:  55 loss_b Variable containing:
 3.9960
[torch.FloatTenso


epoch:  100 loss_b Variable containing:
 3.4651
[torch.FloatTensor of size 1]

epoch:  100 loss_b Variable containing:
 3.5061
[torch.FloatTensor of size 1]

epoch:  100 loss_b Variable containing:
 3.5747
[torch.FloatTensor of size 1]

epoch:  100 loss_b Variable containing:
 3.4633
[torch.FloatTensor of size 1]

epoch:  100 loss_b Variable containing:
 3.4467
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.4253
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.4616
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.5348
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.4596
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.4863
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.5532
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.4134
[torch.FloatTensor of size 1]

epoch:  105 loss_b Variable containing:
 3.4415
[to

epoch:  150 loss_b Variable containing:
 2.9383
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 3.0061
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 3.0095
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 2.9561
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 2.9790
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 3.0038
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 2.8866
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 2.9835
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 3.0397
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 2.8792
[torch.FloatTensor of size 1]

epoch:  150 loss_b Variable containing:
 2.9219
[torch.FloatTensor of size 1]

epoch:  155 loss_b Variable containing:
 2.8812
[torch.FloatTensor of size 1]

epoch:  155 loss_b Variable containing:
 2.9574
[tor

epoch:  195 loss_b Variable containing:
 2.5371
[torch.FloatTensor of size 1]

epoch:  195 loss_b Variable containing:
 2.4251
[torch.FloatTensor of size 1]

epoch:  195 loss_b Variable containing:
 2.5551
[torch.FloatTensor of size 1]

epoch:  195 loss_b Variable containing:
 2.5939
[torch.FloatTensor of size 1]

epoch:  195 loss_b Variable containing:
 2.4469
[torch.FloatTensor of size 1]

epoch:  195 loss_b Variable containing:
 2.4714
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.4898
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.5518
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.5530
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.4689
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.5202
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.4989
[torch.FloatTensor of size 1]

epoch:  200 loss_b Variable containing:
 2.3894
[tor

...

epoch:  290 loss_b Variable containing:
 2.0502
[torch.FloatTensor of size 1]

epoch:  290 loss_b Variable containing:
 1.9893
[torch.FloatTensor of size 1]

epoch:  290 loss_b Variable containing:
 2.0788
[torch.FloatTensor of size 1]

epoch:  290 loss_b Variable containing:
 2.0814
[torch.FloatTensor of size 1]

epoch:  290 loss_b Variable containing:
 1.9653
[torch.FloatTensor of size 1]

epoch:  290 loss_b Variable containing:
 1.9672
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 2.1289
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 2.0671
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 2.1080
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 1.9879
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 2.0569
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 2.0324
[torch.FloatTensor of size 1]

epoch:  295 loss_b Variable containing:
 1.9786
[tor

epoch:  340 loss_b Variable containing:
 2.0192
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.9366
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.9828
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.8640
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.9197
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.9119
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.8832
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.9418
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.9365
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.8234
[torch.FloatTensor of size 1]

epoch:  340 loss_b Variable containing:
 1.7873
[torch.FloatTensor of size 1]

epoch:  345 loss_b Variable containing:
 2.0077
[torch.FloatTensor of size 1]

epoch:  345 loss_b Variable containing:
 1.9228
[tor

epoch:  385 loss_b Variable containing:
 1.8050
[torch.FloatTensor of size 1]

epoch:  385 loss_b Variable containing:
 1.8073
[torch.FloatTensor of size 1]

epoch:  385 loss_b Variable containing:
 1.8330
[torch.FloatTensor of size 1]

epoch:  385 loss_b Variable containing:
 1.8367
[torch.FloatTensor of size 1]

epoch:  385 loss_b Variable containing:
 1.7354
[torch.FloatTensor of size 1]

epoch:  385 loss_b Variable containing:
 1.6537
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.9294
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.8244
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.8651
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.7452
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.7858
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.7951
[torch.FloatTensor of size 1]

epoch:  390 loss_b Variable containing:
 1.7981
[tor

epoch:  435 loss_b Variable containing:
 1.8393
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.7513
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.7648
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.6639
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.6748
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.7027
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.7122
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.7347
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.7348
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.6394
[torch.FloatTensor of size 1]

epoch:  435 loss_b Variable containing:
 1.5110
[torch.FloatTensor of size 1]

epoch:  440 loss_b Variable containing:
 1.8312
[torch.FloatTensor of size 1]

epoch:  440 loss_b Variable containing:
 1.7432
[tor

epoch:  480 loss_b Variable containing:
 1.6353
[torch.FloatTensor of size 1]

epoch:  480 loss_b Variable containing:
 1.6425
[torch.FloatTensor of size 1]

epoch:  480 loss_b Variable containing:
 1.6673
[torch.FloatTensor of size 1]

epoch:  480 loss_b Variable containing:
 1.6466
[torch.FloatTensor of size 1]

epoch:  480 loss_b Variable containing:
 1.5608
[torch.FloatTensor of size 1]

epoch:  480 loss_b Variable containing:
 1.4114
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.7678
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.6763
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.6747
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.5835
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.5896
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.6288
[torch.FloatTensor of size 1]

epoch:  485 loss_b Variable containing:
 1.6348
[tor

In [18]:
idxx = 181

print(target[idxx])

for i in range(283):
    print("------")
    out_numpy = output.data.numpy()
    print("argmax", np.argmax(out_numpy[i]))

Variable containing:
 27
[torch.LongTensor of size 1]

------
argmax 40
------
argmax 21
------
argmax 40
------
argmax 14
------
argmax 17
------
argmax 25
------
argmax 18
------
argmax 18
------
argmax 40
------
argmax 0
------
argmax 2
------
argmax 61
------
argmax 29
------
argmax 22
------
argmax 36
------
argmax 23
------
argmax 48
------
argmax 27
------
argmax 10
------
argmax 27
------
argmax 24
------
argmax 16
------
argmax 21
------
argmax 27
------
argmax 60
------
argmax 56
------
argmax 36
------
argmax 40
------
argmax 27
------
argmax 18
------
argmax 21
------
argmax 22
------
argmax 29
------
argmax 56
------
argmax 18
------
argmax 7
------
argmax 34
------
argmax 23
------
argmax 14
------
argmax 10
------
argmax 24
------
argmax 21
------
argmax 23
------
argmax 18
------
argmax 10
------
argmax 19
------
argmax 18
------
argmax 29
------
argmax 14
------
argmax 24
------
argmax 1
------
argmax 18
------
argmax 13
------
argmax 23
------
argmax 10
------
argmax 